# Parquet - Exercises

**Creating a Table with a Parquet File and Pyarrow**

Like other formats, Apache Parquet is also a valid file format for Hive data. In this exercise, we will convert a CSV file to Parquet and create a table in Hive. We will use the `Stars_Names.csv` file.


## Import the Dataset and create Pandas Dataframe

The first thing we will do is import the file into a Notebook and create a Pandas DataFrame to read its content.  

In [1]:
! mkdir -p /media/notebooks/stars

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [2]:
# Download the csv file inside the folder
! wget https://github.com/Vega90/datasets/raw/main/Stars_Names.csv \
-O /media/notebooks/stars/Stars_Names.csv

--2024-11-10 15:51:07--  https://github.com/Vega90/datasets/raw/main/Stars_Names.csv
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Vega90/datasets/main/Stars_Names.csv [following]
--2024-11-10 15:51:08--  https://raw.githubusercontent.com/Vega90/datasets/main/Stars_Names.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16394 (16K) [text/plain]
Saving to: ‘/media/notebooks/stars/Stars_Names.csv’

/media/notebooks/st 100%[===================>]  16.01K  --.-KB/s    in 0.002s  

2024-11-10 15:51:08 (7.18 MB/s) - ‘/media/notebooks/stars/Stars_Names.csv’ saved [16394/16394]



In [3]:
import pandas as pd

df_stars = pd.read_csv('/media/notebooks/stars/Stars_Names.csv')
df_stars.head()

,Name,Constellation,Bayern Designation,Designation,Approval Date
0,Acamar,Eridanus,θ1 Eridani A,HR 897,2016-07-20
1,Achernar,Eridanus,α Eridani A,HR 472,2016-06-30
2,Achird,Cassiopeia,η Cassiopeiae A,HR 219,2017-09-05
3,Acrab,Scorpius,β1 Scorpii Aa,HR 5984,2016-08-21
4,Acrux,Crux,α Crucis Aa,HR 4730,2016-07-20


Since the column names contain spaces, we will rename the columns.

In [4]:
# Rename columns
df_stars.columns = ['Name', 'Constellation', 'BayernDesignation', 'Designation', 'ApprovalDate']

# Get column types, they are of type object (string)
df_stars.dtypes

Name                 object
Constellation        object
BayernDesignation    object
Designation          object
ApprovalDate         object
dtype: object

## Convert to Parquet

Then, from this file, we will create a Parquet file that we will upload to HDFS and use to create a table in Hive. 

In this case, we will use the PyArrow library, which allows us to define the schema that we will use for our Parquet file. This way, we will have more control and flexibility over the file.

In [5]:
# Create the schema with pyarrow
import pyarrow as pa
import pyarrow.parquet as pq

# Define the schema
esquema = pa.schema([
    ('Name', pa.string()),
    ('Constellation', pa.string()),
    ('BayernDesignation', pa.string()),
    ('Designation', pa.string()),
    ('ApprovalDate', pa.string())
])

In [8]:
# Convert the pandas DataFrame to a PyArrow table with the defined schema
table = pa.Table.from_pandas(df_stars, schema=esquema)

# Convert to Parquet
pq.write_table(table, '/media/notebooks/stars/stars.parq')

# Display parquet file
! cat /media/notebooks/stars/stars.parq | tail

J��ѣH�*]ʴ�ӧ&��5 Designation��@�"&��&�v6 (PSR B1257+12GJ 551 ,      ��L   �4
7-:F2-08 86-10.b 11-06FH11-19
   2018-08-10 ��,�6 (
2018-08-10
2015-12-15   ��   �U2 0C"RP"2 a1"u p �7h&c1 ce`�P#5 UxeaS�2C$ #qq3!4�0@d���<&�#Lcv %��;DRr%SQ  1S@83S� �!B`0TZc332e1"S05Q5Sb �S'bFLR8�b�S0V02&��5 ApprovalDate���&ڜ&ښ6 (
2018-08-10
2015-12-15 ,      l5 schema
Constellation���&�=&�36(	Vulpecula	Andromeda ,      &�t5 BayernDesignation��]�1&�m&�C6(ω Sagittarii A14 Andromedae A ,      &��5 Designation��@�"&��&�v6 (PSR B1257+12GJ 551 ,      &��5 ApprovalDate���&ڜ&ښ6 (
2018-08-10
/////+AEAAAQAAAAAAAKAA4ABgAFAAgACgAAAAABBAAQAAAAAAAKAAwAAAAEAAgACgAAAKADAAAEAAAAAQAAAAwAAAAIAAwABAAIAAgAAAB4AwAABAAAAGgDAAB7ImluZGV4X2NvbHVtbnMiOiBbXSwgImNvbHVtbl9pbmRleGV

Once again, we compare the size of both files and see that the Parquet file is smaller than the CSV file, as in the previous example.


In [9]:
print('The size of the CSV file is:')
! stat -c%s /media/notebooks/stars/Stars_Names.csv

# tamaño del fichero parquet
print('The size of the PARQUET file is:')
! stat -c%s /media/notebooks/stars/stars.parq

The size of the CSV file is:
16394
The size of the PARQUET file is:
13604


## Creating the Stars Table (Hive) from the Parquet File

We need to upload the file we just created to HDFS. We can upload it to `/stars-parquet`, for example. Once done, we will create an external table called `Stars`, but this time we will use the Parquet file we just created.


In [10]:
# Create a dataFrame from the parquet file
dt_parquet = pd.read_parquet('/media/notebooks/stars/stars.parq')
dt_parquet.head()

,Name,Constellation,BayernDesignation,Designation,ApprovalDate
0,Acamar,Eridanus,θ1 Eridani A,HR 897,2016-07-20
1,Achernar,Eridanus,α Eridani A,HR 472,2016-06-30
2,Achird,Cassiopeia,η Cassiopeiae A,HR 219,2017-09-05
3,Acrab,Scorpius,β1 Scorpii Aa,HR 5984,2016-08-21
4,Acrux,Crux,α Crucis Aa,HR 4730,2016-07-20


In [11]:
# Upload to HDFS
! hdfs dfs -mkdir /stars-parquet
! hdfs dfs -put /media/notebooks/stars/stars.parq /stars-parquet

Create a external table:

In [12]:
!beeline -u "jdbc:hive2://" -e \
"CREATE EXTERNAL TABLE starsparquet( \
                 Name STRING, \
                 Constellation STRING, \
                 BayernDesignation STRING, \
                 Designation STRING, \
                 ApprovalDate STRING \
            ) \
            STORED AS PARQUET \
            LOCATION '/stars-parquet';"

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/local/hive/lib/log4j-slf4j-impl-2.18.0.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/local/hadoop/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/local/hive/lib/log4j-slf4j-impl-2.18.0.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/local/hadoop/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Connecting to jdbc:hive2://
Hive Session

The previous command will create an external table that has the data in that location, but in Parquet format. Now, we don’t need to specify the delimiter or indicate that our data has headers. If we run this command in Hive, we can create the table and perform queries as we did in previous modules. For example, we can retrieve the first three records from the table.

In [13]:
!beeline -u "jdbc:hive2://" -e \
"SELECT * FROM starsparquet LIMIT 3;"

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/local/hive/lib/log4j-slf4j-impl-2.18.0.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/local/hadoop/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/local/hive/lib/log4j-slf4j-impl-2.18.0.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/local/hadoop/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Connecting to jdbc:hive2://
Hive Session